# NetBox Demo Charts

Visualizations of network infrastructure data pulled directly from NetBox.

In [ ]:
import os
import pynetbox
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px
import networkx as nx

plt.rcParams.update({
    "figure.facecolor": "#1e1e2e",
    "axes.facecolor": "#1e1e2e",
    "text.color": "#cdd6f4",
    "axes.labelcolor": "#cdd6f4",
    "xtick.color": "#cdd6f4",
    "ytick.color": "#cdd6f4",
    "font.family": "sans-serif",
    "font.size": 11,
})

nb = pynetbox.api(
    os.environ.get("NETBOX_URL", "http://localhost:8000"),
    token=os.environ["NETBOX_API_TOKEN"],
)

devices = list(nb.dcim.devices.all())
prefixes = list(nb.ipam.prefixes.all())
ip_addresses = list(nb.ipam.ip_addresses.all())
cables = list(nb.dcim.cables.all())

df_devices = pd.DataFrame([
    {
        "name": d.name,
        "role": d.role.name if d.role else "Unknown",
        "site": d.site.name if d.site else "Unknown",
        "location": d.location.name if d.location else "No Location",
        "status": d.status.label if hasattr(d.status, "label") else str(d.status),
    }
    for d in devices
])

df_prefixes = pd.DataFrame([
    {
        "prefix": str(p.prefix),
        "mask": int(str(p.prefix).split("/")[1]),
        "total_ips": 2 ** (32 - int(str(p.prefix).split("/")[1])),
        "description": p.description or str(p.prefix),
        "site": p.site.name if p.site else "Global",
        "vlan": p.vlan.name if p.vlan else None,
    }
    for p in prefixes
    if ":" not in str(p.prefix)  # IPv4 only for treemap sizing
])

print(f"Loaded: {len(devices)} devices, {len(prefixes)} prefixes, {len(ip_addresses)} IPs, {len(cables)} cables")

In [ ]:
# --- Device Inventory by Role (Donut Chart) ---

role_counts = df_devices["role"].value_counts()

role_colors = {
    "Router": "#89b4fa",
    "Switch": "#a6e3a1",
    "Host": "#fab387",
    "Edge Router": "#f38ba8",
    "Spine": "#cba6f7",
    "Server": "#f9e2af",
}
colors = [role_colors.get(r, "#585b70") for r in role_counts.index]

fig, ax = plt.subplots(figsize=(8, 8))
wedges, texts, autotexts = ax.pie(
    role_counts.values,
    labels=role_counts.index,
    autopct=lambda pct: f"{int(round(pct / 100 * role_counts.sum()))}",
    colors=colors,
    pctdistance=0.8,
    wedgeprops={"width": 0.4, "edgecolor": "#1e1e2e", "linewidth": 2},
    textprops={"color": "#cdd6f4", "fontsize": 12},
)
for at in autotexts:
    at.set_color("#cdd6f4")
    at.set_fontweight("bold")
    at.set_fontsize(14)

ax.set_title("Device Inventory by Role", fontsize=16, fontweight="bold", pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# --- Site Hierarchy (Stacked Horizontal Bar) ---

site_loc = df_devices.groupby(["site", "location"]).size().unstack(fill_value=0)

loc_palette = [
    "#89b4fa", "#a6e3a1", "#fab387", "#f38ba8",
    "#cba6f7", "#f9e2af", "#94e2d5", "#eba0ac",
    "#74c7ec", "#b4befe",
]

fig, ax = plt.subplots(figsize=(10, 5))
left = pd.Series(0, index=site_loc.index)
for i, loc in enumerate(site_loc.columns):
    color = loc_palette[i % len(loc_palette)]
    ax.barh(site_loc.index, site_loc[loc], left=left, label=loc,
            color=color, edgecolor="#1e1e2e", linewidth=1)
    # Add count labels inside segments
    for y_idx, (val, l) in enumerate(zip(site_loc[loc], left)):
        if val > 0:
            ax.text(l + val / 2, y_idx, str(val),
                    ha="center", va="center", fontweight="bold", fontsize=11, color="#1e1e2e")
    left += site_loc[loc]

ax.set_xlabel("Number of Devices")
ax.set_title("Device Distribution by Site & Location", fontsize=14, fontweight="bold", pad=15)
ax.legend(loc="lower right", framealpha=0.8, facecolor="#313244", edgecolor="#585b70")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_color("#585b70")
ax.spines["left"].set_color("#585b70")
plt.tight_layout()
plt.show()

In [ ]:
# --- IP Address Space (Treemap) ---

def categorize_prefix(prefix, desc):
    desc_lower = (desc or "").lower()
    prefix_str = str(prefix)
    if "management" in desc_lower or "mgmt" in desc_lower or prefix_str.startswith("10.255.255"):
        return "Management"
    if "p2p" in desc_lower or "point-to-point" in desc_lower or "/30" in prefix_str or "/31" in prefix_str:
        return "P2P Links"
    if "loopback" in desc_lower:
        return "Loopbacks"
    if "dmvpn" in desc_lower:
        return "DMVPN"
    if "containerlab" in desc_lower or "clab" in desc_lower or prefix_str.startswith("172.20"):
        return "Containerlab"
    return "LAN Segments"

df_prefixes["category"] = df_prefixes.apply(
    lambda r: categorize_prefix(r["prefix"], r["description"]), axis=1
)

cat_colors = {
    "Management": "#89b4fa",
    "P2P Links": "#a6e3a1",
    "LAN Segments": "#fab387",
    "Loopbacks": "#cba6f7",
    "DMVPN": "#f38ba8",
    "Containerlab": "#f9e2af",
}

fig = px.treemap(
    df_prefixes,
    path=["category", "prefix"],
    values="total_ips",
    color="category",
    color_discrete_map=cat_colors,
    title="IPv4 Address Space by Category",
    hover_data={"description": True, "total_ips": True},
)
fig.update_layout(
    paper_bgcolor="#1e1e2e",
    plot_bgcolor="#1e1e2e",
    font_color="#cdd6f4",
    title_font_size=16,
    margin=dict(t=50, l=10, r=10, b=10),
    width=900,
    height=550,
)
fig.update_traces(textinfo="label+value", textfont_size=12)
fig.show()

In [ ]:
# --- Network Topology Graph ---

G = nx.Graph()

device_roles = {d.name: (d.role.name if d.role else "Unknown") for d in devices}

for cable in cables:
    a_terms = cable.a_terminations
    b_terms = cable.b_terminations
    if not a_terms or not b_terms:
        continue
    a_ep = a_terms[0]
    b_ep = b_terms[0]
    a_device = getattr(getattr(a_ep, 'device', None), 'name', None)
    b_device = getattr(getattr(b_ep, 'device', None), 'name', None)
    if a_device and b_device:
        a_intf = getattr(a_ep, 'name', '')
        b_intf = getattr(b_ep, 'name', '')
        G.add_edge(a_device, b_device, label=f"{a_intf} - {b_intf}")

node_role_colors = {
    "Router": "#89b4fa",
    "Switch": "#a6e3a1",
    "Host": "#fab387",
    "Edge Router": "#f38ba8",
    "Spine": "#cba6f7",
    "Server": "#f9e2af",
}
node_colors = [node_role_colors.get(device_roles.get(n, ""), "#585b70") for n in G.nodes()]
node_sizes = [300 + 200 * G.degree(n) for n in G.nodes()]

fig, ax = plt.subplots(figsize=(14, 10))
pos = nx.spring_layout(G, seed=42, k=2.5)
nx.draw_networkx_edges(G, pos, ax=ax, edge_color="#585b70", width=1.5, alpha=0.7)
nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_colors, node_size=node_sizes,
                       edgecolors="#cdd6f4", linewidths=1.5)
nx.draw_networkx_labels(G, pos, ax=ax, font_size=9, font_color="#cdd6f4", font_weight="bold")

# Legend
seen_roles = set(device_roles.get(n, "Unknown") for n in G.nodes())
legend_handles = [
    plt.Line2D([0], [0], marker="o", color="#1e1e2e", markerfacecolor=node_role_colors.get(r, "#585b70"),
               markersize=10, label=r)
    for r in sorted(seen_roles) if r in node_role_colors
]
ax.legend(handles=legend_handles, loc="upper left", framealpha=0.8,
          facecolor="#313244", edgecolor="#585b70")

ax.set_title("Network Topology (from NetBox Cables)", fontsize=16, fontweight="bold", pad=20)
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# --- Prefix Utilization (Horizontal Bar) ---

# Count assigned IPs per prefix
prefix_ip_count = {}
for ip in ip_addresses:
    ip_str = str(ip.address)  # e.g. "10.0.0.1/24"
    # Find which prefix this IP belongs to
    from ipaddress import ip_network, ip_address as ip_addr
    try:
        addr = ip_addr(ip_str.split("/")[0])
    except ValueError:
        continue
    if addr.version != 4:
        continue
    for p in prefixes:
        p_str = str(p.prefix)
        if ":" in p_str:
            continue
        try:
            net = ip_network(p_str, strict=False)
        except ValueError:
            continue
        if addr in net:
            prefix_ip_count[p_str] = prefix_ip_count.get(p_str, 0) + 1

# Build utilization dataframe (only prefixes with assigned IPs)
util_rows = []
for p_str, count in sorted(prefix_ip_count.items()):
    mask = int(p_str.split("/")[1])
    total = 2 ** (32 - mask)
    # For /31 and /32, usable == total; otherwise subtract network + broadcast
    usable = total if mask >= 31 else max(total - 2, 1)
    util = min(count / usable, 1.0)
    util_rows.append({"prefix": p_str, "assigned": count, "usable": usable, "utilization": util})

df_util = pd.DataFrame(util_rows).sort_values("utilization", ascending=True)

# Color gradient: green -> yellow -> red
cmap = mcolors.LinearSegmentedColormap.from_list("util", ["#a6e3a1", "#f9e2af", "#f38ba8"])
bar_colors = [cmap(u) for u in df_util["utilization"]]

fig, ax = plt.subplots(figsize=(10, max(4, len(df_util) * 0.45)))
bars = ax.barh(df_util["prefix"], df_util["utilization"], color=bar_colors,
               edgecolor="#1e1e2e", linewidth=1)

for bar, (_, row) in zip(bars, df_util.iterrows()):
    pct = f"{row['utilization']:.0%}"
    detail = f"  {pct}  ({row['assigned']}/{row['usable']})"
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height() / 2,
            detail, va="center", fontsize=10, color="#cdd6f4")

ax.set_xlim(0, 1.35)
ax.set_xlabel("Utilization")
ax.set_title("Prefix Utilization (Assigned IPs / Usable IPs)", fontsize=14, fontweight="bold", pad=15)
ax.axvline(x=0.8, color="#f38ba8", linestyle="--", alpha=0.5, label="80% threshold")
ax.legend(loc="lower right", framealpha=0.8, facecolor="#313244", edgecolor="#585b70")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_color("#585b70")
ax.spines["left"].set_color("#585b70")
plt.tight_layout()
plt.show()